In [1]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
import wandb

from datasets import Dataset, load_dataset
from trl import SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, TrainingArguments

wandb.finish()
os.environ['WANDB_DISABLED'] = 'true'

# dataset = load_dataset("imdb", split="train")

# if torch.backends.mps.is_built():
#     device = torch.device("mps")

model_name = 'AnatoliiPotapov/T-lite-instruct-0.1'

In [8]:
# df = pd.read_csv('financial_dialogue_dataset.csv')
# texts0 = df.to_dict(orient='records')
# texts0[:5]

In [3]:
texts = [
    {"role": "user", "content": "Каковы текущие котировки акций компании Apple?"},
    {"role": "bot", "content": "Текущие котировки акций Apple составляют $173.21."},
    
    {"role": "user", "content": "Какая доходность облигаций государственного займа США на сегодня?"},
    {"role": "bot", "content": "Доходность облигаций государственного займа США на сегодня составляет 2.75%."},
    
    {"role": "user", "content": "Расскажи о последних новостях на фондовом рынке."},
    {"role": "bot", "content": "Сегодня на фондовом рынке наблюдается рост индексов S&P 500 и NASDAQ после позитивных экономических отчетов."},
    
    {"role": "user", "content": "Как изменился курс евро к доллару за последние неделю?"},
    {"role": "bot", "content": "Курс евро к доллару изменился с 1.092 до 1.095 за последнюю неделю."},
    
    {"role": "user", "content": "Какие акции показали наибольший рост в этом месяце?"},
    {"role": "bot", "content": "В этом месяце наибольший рост показали акции Tesla, выросшие на 15%."},
    
    {"role": "user", "content": "Какие компании выплатят дивиденды в этом квартале?"},
    {"role": "bot", "content": "В этом квартале дивиденды выплатят компании Microsoft, Johnson & Johnson и Coca-Cola."},
    
    {"role": "user", "content": "Каков прогноз по цене золота на следующий месяц?"},
    {"role": "bot", "content": "Прогноз по цене золота на следующий месяц предполагает её рост до $1,950 за унцию."},
    
    {"role": "user", "content": "Какие экономические отчеты ожидаются на этой неделе?"},
    {"role": "bot", "content": "На этой неделе ожидаются отчеты по безработице и ВВП США."},
    
    {"role": "user", "content": "Расскажи о финансовых результатах компании Microsoft за последний квартал."},
    {"role": "bot", "content": "Компания Microsoft сообщила о доходах в размере $55.5 млрд и чистой прибыли $18.3 млрд за последний квартал."},
    
    {"role": "user", "content": "Как изменились ставки по ипотечным кредитам в этом месяце?"},
    {"role": "bot", "content": "Ставки по ипотечным кредитам в этом месяце снизились до 4.20%."},
    
    {"role": "user", "content": "Какие активы лучше всего инвестировать в условиях нестабильной экономики?"},
    {"role": "bot", "content": "В условиях нестабильной экономики рекомендуется инвестировать в золото, государственные облигации и диверсифицированные инвестиционные фонды."},
    
    {"role": "user", "content": "Что такое криптовалюта и как она работает?"},
    {"role": "bot", "content": "Криптовалюта — это цифровая валюта, использующая криптографию для обеспечения безопасности транзакций. Работает на основе блокчейн-технологии."},
    
    {"role": "user", "content": "Какие акции торгуются на Нью-Йоркской фондовой бирже?"},
    {"role": "bot", "content": "На Нью-Йоркской фондовой бирже торгуются такие акции, как IBM, Pfizer, Bank of America и многие другие."},
    
    {"role": "user", "content": "Какой прогноз по цене нефти на следующий квартал?"},
    {"role": "bot", "content": "Прогноз по цене нефти на следующий квартал предполагает её снижение до $70 за баррель."},
    
    {"role": "user", "content": "Какой текущий курс биткойна к доллару США?"},
    {"role": "bot", "content": "Текущий курс биткойна к доллару США составляет $27,500."},
    
    {"role": "user", "content": "Какие факторы влияют на изменение валютного курса?"},
    {"role": "bot", "content": "На изменение валютного курса влияют экономические показатели, политические события, рыночные ожидания и действия центральных банков."},
    
    {"role": "user", "content": "Какие инвестиции лучше всего подходят для пенсионного плана?"},
    {"role": "bot", "content": "Для пенсионного плана подойдут диверсифицированные инвестиционные фонды, облигации и высокодоходные акции компаний с долгой историей выплат дивидендов."},
    
    {"role": "user", "content": "Какие новости из Китая могут повлиять на мировую экономику?"},
    {"role": "bot", "content": "Новости из Китая, такие как изменения в экономической политике, данные по ВВП или торговые соглашения, могут значительно повлиять на мировую экономику."},
    
    {"role": "user", "content": "Каков текущий тренд на рынке недвижимости?"},
    {"role": "bot", "content": "Текущий тренд на рынке недвижимости показывает рост цен на жилье в крупных городах и замедление темпов роста в пригородах."},
    
    {"role": "user", "content": "Какие риски связаны с инвестированием в стартапы?"},
    {"role": "bot", "content": "Риски инвестирования в стартапы включают высокий уровень неопределенности, отсутствие доходов на начальных этапах и потенциальные проблемы с управлением компанией."},
    
    {"role": "user", "content": "Как выбрать хорошего финансового консультанта?"},
    {"role": "bot", "content": "Для выбора хорошего финансового консультанта важно учитывать его опыт, репутацию, уровень сертификации и подход к управлению инвестициями."},
    
    {"role": "user", "content": "Как изменяются фондовые индексы в зависимости от экономических условий?"},
    {"role": "bot", "content": "Фондовые индексы могут расти в условиях экономического роста и снижения, и наоборот, падать в условиях экономического спада."}
]

In [5]:
# Преобразование в формат id, source, message
formatted_texts = []
id_counter = 1

for i in range(0, len(texts), 2):
    user_message = texts[i]['content']
    bot_message = texts[i + 1]['content']
    
    formatted_texts.append({
        "id": id_counter,
        "source": "user",
        "message": user_message
    })
    
    formatted_texts.append({
        "id": id_counter,
        "source": "bot",
        "message": bot_message
    })
    
    id_counter += 1

In [6]:
df = pd.DataFrame(
    {
        'id': [item['id'] for item in formatted_texts],
        'source': [item['source'] for item in formatted_texts],
        'message': [item['message'] for item in formatted_texts]
    }
)
hg_dataset = Dataset(pa.Table.from_pandas(df))

In [7]:
# !pip install trl # PEFT
# !pip install git+https://github.com/huggingface/peft.git # PEFT
# !pip install -U git+https://github.com/huggingface/transformers.git # flash-attention

import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
import wandb

from datasets import Dataset, load_dataset
from peft import LoraConfig
from trl import ModelConfig, SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, TrainingArguments

model_name = 'AnatoliiPotapov/T-lite-instruct-0.1'

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

model_config = ModelConfig(
    model_name_or_path=model_name,
    attn_implementation='flash_attention_2'
)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

sft_config = TrainingArguments(
    output_dir='test_trainer',
    per_device_train_batch_size=4,  # Убедитесь, что размер батча подходит для вашей памяти GPU
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # Пример числа эпох, настройте по своему усмотрению
    logging_dir='logs',
    logging_steps=10,
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=hg_dataset,
    args=sft_config,
    dataset_text_field="message",
    packing=False,
    peft_config=peft_config,
    max_seq_length=256
)

trainer.train()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/user1/environments/hack/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/user1/environments/hack/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/user1/environments/hack/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, t

Map:   0%|          | 0/1044 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 31.73 GiB of which 1.42 GiB is free. Including non-PyTorch memory, this process has 30.31 GiB memory in use. Of the allocated memory 29.94 GiB is allocated by PyTorch, and 4.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)